In [1]:
from bs4 import BeautifulSoup
import cloudscraper
import json
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import time, random

In [2]:
blacklist = {"https://batdongsan.com.vn/ban-dat-ha-noi/gia-tu-800-trieu-den-1-ty",
             "https://batdongsan.com.vn/ban-nha-rieng-ha-noi/gia-tu-1-ty-den-2-ty",
             "https://batdongsan.com.vn/ban-nha-mat-pho-ha-noi/gia-tu-2-ty-den-3-ty",
             "https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-ha-noi/gia-tu-3-ty-den-5-ty",
             "https://batdongsan.com.vn/ban-can-ho-chung-cu-ha-noi", "https://batdongsan.com.vn/ban-nha-mat-pho-ha-noi",
             "https://batdongsan.com.vn/ban-dat-ha-noi", "https://batdongsan.com.vn/ban-nha-rieng-hoai-duc",
             "https://batdongsan.com.vn/ban-nha-rieng-thanh-xuan", "https://batdongsan.com.vn/ban-nha-rieng-hoang-mai",
             "https://batdongsan.com.vn/ban-nha-rieng-nam-tu-liem", "https://batdongsan.com.vn/ban-nha-rieng",
             "https://batdongsan.com.vn/ban-nha-rieng-ha-dong", "https://batdongsan.com.vn/ban-nha-rieng-cau-giay",
             "https://batdongsan.com.vn/ban-nha-rieng-hai-ba-trung", "https://batdongsan.com.vn/ban-nha-rieng-ha-noi",
             "https://batdongsan.com.vn/ban-nha-rieng-dong-da", "https://batdongsan.com.vn/ban-nha-rieng-bac-tu-liem",
             "https://batdongsan.com.vn/ban-nha-rieng-ba-dinh", "https://batdongsan.com.vn/ban-nha-rieng-long-bien",
             "https://batdongsan.com.vn/ban-nha-rieng-tay-ho"}

In [9]:
def get_all_bds_links_titles(start_page=1, end_page=2000, max_workers=10, save_every=1):
    all_results = []
    def crawl_page(i):
        time.sleep(random.uniform(2, 4))
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36')
        prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.default_content_setting_values.notifications": 2,
            "profile.managed_default_content_settings.stylesheets": 2,
            "profile.managed_default_content_settings.fonts": 2,
        }
        options.add_experimental_option("prefs", prefs)
        driver = webdriver.Chrome(options=options)
        url = f"https://batdongsan.com.vn/nha-dat-ban-ha-noi/p{i}"
        page_results = []
        try:
            driver.get(url)
            time.sleep(random.uniform(2, 3))
            links = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/ban-"]')
            for a in links:
                href = a.get_attribute('href')
                title = a.get_attribute('title')
                if not title:
                    try:
                        span_title = a.find_element(By.CSS_SELECTOR, 'span.js__card-title, span.vip-diamond.js__card-title')
                        title = span_title.text.strip()
                    except:
                        title = None
                if href and href in blacklist:
                    continue
                if title and href:
                    page_results.append({'title': title, 'detail_link': href})
            print(f"Đã lấy xong page {i}, số link: {len(page_results)}")
        except Exception as e:
            print(f"Lỗi ở page {i}: {e}")
        driver.quit()
        return page_results
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(crawl_page, i) for i in range(start_page, end_page+1)]
        for idx, future in enumerate(as_completed(futures)):
            page_links = future.result()
            if page_links:
                all_results.extend(page_links)
            # Lọc trùng ngay sau mỗi lần thêm trang mới
            unique_links = {}
            for item in all_results:
                unique_links[item['detail_link']] = item
            all_results = list(unique_links.values())
            # Lưu file mỗi save_every lần hoặc lần cuối
            if (idx+1) % save_every == 0 or (idx+1) == (end_page-start_page+1):
                with open('data/batdongsan_links.json', 'w', encoding='utf-8') as f:
                    json.dump(all_results, f, ensure_ascii=False, indent=2)
            print(f"Tiến độ lấy link: {idx+1}/{end_page-start_page+1}")
    print(f"Tổng số link duy nhất: {len(all_results)}")
    return all_results

In [10]:
# Lấy tất cả link và tiêu đề từ batdongsan
results = get_all_bds_links_titles(start_page=1, end_page=2000, max_workers=10, save_every=10)

Đã lấy xong page 9, số link: 29
Đã lấy xong page 1, số link: 29
Đã lấy xong page 4, số link: 29
Đã lấy xong page 3, số link: 29
Đã lấy xong page 7, số link: 29
Đã lấy xong page 2, số link: 29
Đã lấy xong page 6, số link: 29
Đã lấy xong page 8, số link: 29
Đã lấy xong page 5, số link: 29
Đã lấy xong page 10, số link: 29
Tiến độ lấy link: 1/2000
Tiến độ lấy link: 2/2000
Tiến độ lấy link: 3/2000
Tiến độ lấy link: 4/2000
Tiến độ lấy link: 5/2000
Tiến độ lấy link: 6/2000
Tiến độ lấy link: 7/2000
Tiến độ lấy link: 8/2000
Tiến độ lấy link: 9/2000
Tiến độ lấy link: 10/2000
Đã lấy xong page 11, số link: 29
Đã lấy xong page 13, số link: 29
Đã lấy xong page 15, số link: 29
Đã lấy xong page 12, số link: 29
Đã lấy xong page 14, số link: 29
Đã lấy xong page 16, số link: 29
Đã lấy xong page 18, số link: 29
Đã lấy xong page 19, số link: 29
Tiến độ lấy link: 11/2000
Tiến độ lấy link: 12/2000
Tiến độ lấy link: 13/2000
Tiến độ lấy link: 14/2000
Tiến độ lấy link: 15/2000
Tiến độ lấy link: 16/2000
Tiến độ 

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json)); using driver found in the cache
There was an error managing chrome; using browser found in the cache


Tiến độ lấy link: 59/2000
Đã lấy xong page 65, số link: 29
Đã lấy xong page 55, số link: 29
Đã lấy xong page 69, số link: 29
Đã lấy xong page 62, số link: 29
Tiến độ lấy link: 60/2000
Tiến độ lấy link: 61/2000
Tiến độ lấy link: 62/2000
Tiến độ lấy link: 63/2000
Đã lấy xong page 70, số link: 29
Đã lấy xong page 63, số link: 29
Đã lấy xong page 64, số link: 29
Đã lấy xong page 56, số link: 29
Đã lấy xong page 71, số link: 29
Đã lấy xong page 66, số link: 29
Đã lấy xong page 72, số link: 29
Đã lấy xong page 67, số link: 29
Tiến độ lấy link: 64/2000
Tiến độ lấy link: 65/2000
Tiến độ lấy link: 66/2000
Đã lấy xong page 68, số link: 29
Tiến độ lấy link: 67/2000
Tiến độ lấy link: 68/2000
Tiến độ lấy link: 69/2000
Tiến độ lấy link: 70/2000
Tiến độ lấy link: 71/2000
Tiến độ lấy link: 72/2000
Đã lấy xong page 74, số link: 29
Đã lấy xong page 76, số link: 29
Đã lấy xong page 78, số link: 29
Tiến độ lấy link: 73/2000
Đã lấy xong page 73, số link: 30
Tiến độ lấy link: 74/2000
Tiến độ lấy link: 75/20

In [ ]:
    # <div class="re__pr-info pr-info js__product-detail-web" id="product-detail-web" uid="4454141" prid="43119742">
    #     <h1 class="re__pr-title pr-title js__pr-title">Cần bán nhanh 3 suất ngoại giao biệt thự, liền kề - VIP nhất Vinhomes Đan Phượng (WDC)</h1>
    #     <span class="re__pr-short-description js__pr-address">Vinhomes Wonder City, Tân Hội, Đan Phượng, Hà Nội</span>
    #     <div class="re__section re__pr-specs re__pr-specs-v1 js__section js__li-specs">
    #         <span class="re__section-title">Đặc điểm bất động sản</span>
    #         <div class="re__section-body re__border--std js__section-body">
    #             <div class="re__pr-specs-content-v2 js__other-info">
    # <div class="re__pr-other-info-display">
    # <div class="re__pr-specs-content-item">
    #     <span class="re__pr-specs-content-item-icon"><i class="re__icon-money"></i></span>
    #     <span class="re__pr-specs-content-item-title">Mức giá</span>
    #     <span class="re__pr-specs-content-item-value">2,3 t&#x1EF7;</span>
    # </div>
    # <div class="re__pr-specs-content-item">
    #     <span class="re__pr-specs-content-item-icon"><i class="re__icon-size"></i></span>
    #     <span class="re__pr-specs-content-item-title">Diện tích</span>
    #     <span class="re__pr-specs-content-item-value">100 m&#xB2;</span>
    # </div>
    # <div class="re__pr-specs-content-item">
    #     <span class="re__pr-specs-content-item-icon"><i class="re__icon-front-view"></i></span>
    #     <span class="re__pr-specs-content-item-title">Hướng nhà</span>
    #     <span class="re__pr-specs-content-item-value">Nam</span>
    # </div>
    # <div class="re__pr-specs-content-item">
    #     <span class="re__pr-specs-content-item-icon"><i class="re__icon-home"></i></span>
    #     <span class="re__pr-specs-content-item-title">Mặt tiền</span>
    #     <span class="re__pr-specs-content-item-value">5,5 m</span>
    # </div>
    # <div class="re__pr-specs-content-item">
    #     <span class="re__pr-specs-content-item-icon"><i class="re__icon-road"></i></span>
    #     <span class="re__pr-specs-content-item-title">Đường vào</span>
    #     <span class="re__pr-specs-content-item-value">5 m</span>
    # </div>
    # <div class="re__pr-specs-content-item">
    #     <span class="re__pr-specs-content-item-icon"><i class="re__icon-document"></i></span>
    #     <span class="re__pr-specs-content-item-title">Pháp lý</span>
    #     <span class="re__pr-specs-content-item-value">Sổ đỏ/ Sổ hồng.</span>

In [39]:
def crawl_bds_detail_selenium(url):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    result = {'detail_link': url}
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.re__pr-title'))
        )
        h1 = driver.find_element(By.CSS_SELECTOR, 'h1.re__pr-title')
        result['Tiêu đề'] = h1.text.strip()
    except:
        pass
    try:
        address_block = driver.find_element(By.CSS_SELECTOR, 'span.re__pr-short-description')
        result['Địa chỉ'] = address_block.text.strip()
    except:
        pass
    try:
        specs = driver.find_elements(By.CSS_SELECTOR, 'div.re__pr-specs-content-item')
        for spec in specs:
            try:
                key = spec.find_element(By.CSS_SELECTOR, 'span.re__pr-specs-content-item-title').text.strip()
                value = spec.find_element(By.CSS_SELECTOR, 'span.re__pr-specs-content-item-value').text.strip()
                result[key] = value
            except:
                continue
    except:
        pass
    try:
        phone_block = driver.find_element(By.CSS_SELECTOR, 'div.InlineShowPhoneButton_linkContact__U_lEr')
        result['SĐT'] = phone_block.text.strip()
    except:
        pass
    driver.quit()
    return result

In [40]:
# Lọc link trùng theo detail_link
unique_links = {}
for item in results:
    unique_links[item['detail_link']] = item
results = list(unique_links.values())
print(f"Số link duy nhất: {len(results)}")

# Đa luồng crawl chi tiết
all_data = []
def crawl_and_collect(item):
    url = item['detail_link']
    try:
        data = crawl_bds_detail_selenium(url)
        print(f"Đã crawl: {url}")
        print(data)
        return data
    except Exception as e:
        print(f"Lỗi khi crawl {url}: {e}")
        return None

max_workers = 20  # Tùy máy, có thể tăng lên 6-8 nếu RAM khỏe
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(crawl_and_collect, item) for item in results]
    for idx, future in enumerate(as_completed(futures)):
        data = future.result()
        if data:
            all_data.append(data)
        print(f"Tiến độ: {idx+1}/{len(results)}")

with open('data/batdongsan_details.json', 'w', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)
print(f"Đã lưu {len(all_data)} kết quả vào data/batdongsan_details.json")

Đã crawl 1/493: https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-hoa-sua-phuong-phuc-loi-prj-vinhomes-riverside/ban-sieu-don-lap-goc-270ty-dien-tich-606m2-co-ong-luong-pr43123789
Đã crawl 2/493: https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-xa-da-ton-prj-the-beverly-vinhomes-ocean-park/shop-chan-de-chung-cu-ngay-mat-sanh-so-hong-lau-dai-gia-chi-10-15-ty-san-khach-thue-pr43060294
Đã crawl 3/493: https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-xa-duong-xa-prj-vinhomes-ocean-park-gia-lam/37-ty-song-lap-ngoc-trai-150m2-hoan-thien-4pn-co-thang-may-lam-pr43038580
Đã crawl 4/493: https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-ngoc-thuy-phuong-ngoc-thuy/ban-phap-ven-song-hong-dt-1329m2-noi-that-5-sao-view-song-hong-dep-mong-mo-dang-cap-pr40256201
Đã crawl 5/493: https://batdongsan.com.vn/ban-dat-xa-binh-yen-1/chinh-chu-ban-lo-sieu-dep-truc-chinh-thon-canh-chu-phu-thach-that-ha-noi-pr43037783
Đã crawl 6/493: https://batdongsan.com.vn/ban-dat-duong-35-xa-minh-ph